In [1]:
import librosa
import librosa.feature
import librosa.display
import os
import numpy as np
import pandas as pd
from PIL import Image

def load_fixed_audio(path, duration=30, sr=22050):
    audio, _ = librosa.load(path, sr=sr, duration=duration)
    desired_length = duration * sr
    
    # Pad with zeros if shorter
    if len(audio) < desired_length:
        audio = np.pad(audio, (0, desired_length - len(audio)))
    # Or truncate if longer
    else:
        audio = audio[:desired_length]
    
    return audio

def audio_to_melspec(audio, sr=22050, n_mels=128, hop_length=512):
    melspec = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=n_mels, hop_length=hop_length)
    melspec_db = librosa.power_to_db(melspec+1e-10, ref=np.max, amin=1e-10)
    return melspec_db

def normalize(melspec_db):
    min_val = np.min(melspec_db)
    max_val = np.max(melspec_db)
    # Ensure no division by zero
    if max_val - min_val == 0:
        return np.zeros_like(melspec_db)
    return (melspec_db - min_val) / (max_val - min_val)

def get_all_wav_files(directory):
    wav_files = []
    for dirpath, _, filenames in os.walk(directory):
        for filename in filenames:
            if filename.endswith('.wav'):
                wav_files.append(dirpath + '/' + filename)
    return wav_files


def get_all_image_files(directory):
    wav_files = []
    for dirpath, _, filenames in os.walk(directory):
        for filename in filenames:
            if filename.endswith('.png'):
                wav_files.append(dirpath + '/' + filename)
    return wav_files




In [42]:
files = get_all_wav_files("../data/genres")
entries = []

def clean_labels(s):
    return s.split('.')[0]

def extract_features(file_path, sr=22050, n_mfcc=20):
    
    # Load
    y, sr = librosa.load(file_path, sr=sr, mono=True, duration=30)  # ensuring 30s duration
    features = []
    
    # MFCCs
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = mfcc.mean(axis=1)
    mfcc_std = mfcc.std(axis=1)
    features.extend(mfcc_mean)
    features.extend(mfcc_std)
    
    # Chroma features
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    chroma_mean = chroma.mean(axis=1)
    chroma_std = chroma.std(axis=1)
    features.extend(chroma_mean)
    features.extend(chroma_std)
    
    # 3. Spectral contrast
    spec_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    spec_contrast_mean = spec_contrast.mean(axis=1)
    spec_contrast_std = spec_contrast.std(axis=1)
    features.extend(spec_contrast_mean)
    features.extend(spec_contrast_std)
    
    # 4. Tonnetz features (need harmonic component of audio for tonnetz)
    y_harm = librosa.effects.harmonic(y)  # isolate harmonic component
    tonnetz = librosa.feature.tonnetz(y=y_harm, sr=sr)
    tonnetz_mean = tonnetz.mean(axis=1)
    tonnetz_std = tonnetz.std(axis=1)
    features.extend(tonnetz_mean)
    features.extend(tonnetz_std)
    
    # 5. Tempo 
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    features.append(tempo[0])
    
    # 6. Spectral Rolloff
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
    features.append(spectral_rolloff.mean())
    features.append(spectral_rolloff.std())
    
    # 7. Spectral Centroids
    spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    features.append(spectral_centroids.mean())
    features.append(spectral_centroids.std())
    
    # 9. Zero Crossings Rate
    zc = librosa.zero_crossings(y=y, pad=False)
    features.append(sum(zc)/len(zc))
    
    return features

for file in files:
    try:
        print(file)
        filename = file.split('/')[-1]
        row = extract_features(file)
        entries.append([clean_labels(filename)] + row)
    except Exception as e:
        print(e)
        print(f"Could not process {file}")
        
def get_classical_feature_names(n_mfcc=20, n_chroma=12, n_spec_contrast=7, n_tonnetz=6):
    """
    Returns a list of feature names for the classical features: 
    MFCCs, Chroma, Spectral Contrast, Tonnetz, Tempo.
    """
    column_names = ['target']
    
    # MFCC mean & std
    for i in range(1, n_mfcc+1):
        column_names.append(f"mfcc_{i}_mean")
    for i in range(1, n_mfcc+1):
        column_names.append(f"mfcc_{i}_std")
    
    # Chroma mean & std
    for i in range(1, n_chroma+1):
        column_names.append(f"chroma_{i}_mean")
    for i in range(1, n_chroma+1):
        column_names.append(f"chroma_{i}_std")

    # Spectral contrast mean & std
    for i in range(1, n_spec_contrast+1):
        column_names.append(f"spec_contrast_{i}_mean")
    for i in range(1, n_spec_contrast+1):
        column_names.append(f"spec_contrast_{i}_std")

    # Tonnetz mean & std
    for i in range(1, n_tonnetz+1):
        column_names.append(f"tonnetz_{i}_mean")
    for i in range(1, n_tonnetz+1):
        column_names.append(f"tonnetz_{i}_std")
    
    # Tempo
    column_names.append("tempo")
    column_names.append("mean_spectral_rolloff")
    column_names.append("std_spectral_rolloff")
    column_names.append("mean_spectral_centroids")
    column_names.append("std_spectral_centroids")
    
    # Zero Crossing Rate
    column_names.append("zc_rate")
    
    return column_names
        
df = pd.DataFrame(np.row_stack(entries), columns=get_classical_feature_names())
df.to_csv("../data/files/audio_classification.csv")

../data/genres\blues/blues.00000.wav
../data/genres\blues/blues.00001.wav
../data/genres\blues/blues.00002.wav


KeyboardInterrupt: 

In [3]:
files = get_all_wav_files("../data/genres")
entries = []

def clean_labels(s):
    return s.split('.')[0]

def extract_features(file_path,sr=44000,total_duration=30.0,segment_duration=5,n_mfcc=13):
    
    # Load
    y, sr = librosa.load(file_path, sr=sr, mono=True, duration=30)
    
    num_samples = len(y)

    # 2. Calculate how many segments we can extract
    samples_per_segment = int(segment_duration * sr)
    num_segments = int(np.floor(num_samples / samples_per_segment))
    
    # 3. For each segment, extract features
    all_segments = []
    for i in range(num_segments):
        features = []
        start = i * samples_per_segment
        end = start + samples_per_segment
        segment_y = y[start:end]
        
        # MFCCs
        mfcc = librosa.feature.mfcc(y=segment_y, sr=sr, n_mfcc=n_mfcc)
        mfcc_mean = mfcc.mean(axis=1)
        mfcc_std = mfcc.std(axis=1)
        features.extend(mfcc_mean)
        features.extend(mfcc_std)
        
        # Chroma features
        chroma = librosa.feature.chroma_stft(y=segment_y, sr=sr)
        chroma_mean = chroma.mean(axis=1)
        chroma_std = chroma.std(axis=1)
        features.extend(chroma_mean)
        features.extend(chroma_std)
        
        # 3. Spectral contrast
        spec_contrast = librosa.feature.spectral_contrast(y=segment_y, sr=sr)
        spec_contrast_mean = spec_contrast.mean(axis=1)
        spec_contrast_std = spec_contrast.std(axis=1)
        features.extend(spec_contrast_mean)
        features.extend(spec_contrast_std)
        
        # 4. Tonnetz features (need harmonic component of audio for tonnetz)
        y_harm = librosa.effects.harmonic(segment_y)  # isolate harmonic component
        tonnetz = librosa.feature.tonnetz(y=y_harm, sr=sr)
        tonnetz_mean = tonnetz.mean(axis=1)
        tonnetz_std = tonnetz.std(axis=1)
        features.extend(tonnetz_mean)
        features.extend(tonnetz_std)
        
        # 5. Tempo 
        tempo, _ = librosa.beat.beat_track(y=segment_y, sr=sr)
        features.append(tempo[0])
        
        # 6. Spectral Rolloff
        spectral_rolloff = librosa.feature.spectral_rolloff(y=segment_y, sr=sr)[0]
        features.append(spectral_rolloff.mean())
        features.append(spectral_rolloff.std())
        
        # 7. Spectral Centroids
        spectral_centroids = librosa.feature.spectral_centroid(y=segment_y, sr=sr)[0]
        features.append(spectral_centroids.mean())
        features.append(spectral_centroids.std())
        
        # 9. Zero Crossings Rate
        zc = librosa.zero_crossings(y=segment_y, pad=False)
        features.append(sum(zc)/len(zc))
        
        rms = librosa.feature.rms(y=segment_y)
        features.append(rms.mean())
        features.append(rms.std())
        
        all_segments.append(features)
    return all_segments

for file in files:
    try:
        print(file)
        filename = file.split('/')[-1]
        row = extract_features(file)
        for entry in row:
            entries.append([clean_labels(filename)] + [entry])
    except Exception as e:
        print(e)
        print(f"Could not process {file}")
        

        


../data/genres\blues/blues.00000.wav
../data/genres\blues/blues.00001.wav
../data/genres\blues/blues.00002.wav
../data/genres\blues/blues.00003.wav
../data/genres\blues/blues.00004.wav
../data/genres\blues/blues.00005.wav
../data/genres\blues/blues.00006.wav
../data/genres\blues/blues.00007.wav
../data/genres\blues/blues.00008.wav
../data/genres\blues/blues.00009.wav
../data/genres\blues/blues.00010.wav
../data/genres\blues/blues.00011.wav
../data/genres\blues/blues.00012.wav
../data/genres\blues/blues.00013.wav
../data/genres\blues/blues.00014.wav
../data/genres\blues/blues.00015.wav
../data/genres\blues/blues.00016.wav
../data/genres\blues/blues.00017.wav
../data/genres\blues/blues.00018.wav
../data/genres\blues/blues.00019.wav
../data/genres\blues/blues.00020.wav
../data/genres\blues/blues.00021.wav
../data/genres\blues/blues.00022.wav
../data/genres\blues/blues.00023.wav
../data/genres\blues/blues.00024.wav
../data/genres\blues/blues.00025.wav
../data/genres\blues/blues.00026.wav
.

C:\Users\awast\AppData\Local\Temp\ipykernel_39132\2724620927.py:10: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=sr, mono=True, duration=30)
C:\Users\awast\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


../data/genres\jazz/jazz.00056.wav
../data/genres\jazz/jazz.00057.wav
../data/genres\jazz/jazz.00058.wav
../data/genres\jazz/jazz.00059.wav
../data/genres\jazz/jazz.00060.wav
../data/genres\jazz/jazz.00061.wav
../data/genres\jazz/jazz.00062.wav
../data/genres\jazz/jazz.00063.wav
../data/genres\jazz/jazz.00064.wav
../data/genres\jazz/jazz.00065.wav
../data/genres\jazz/jazz.00066.wav
../data/genres\jazz/jazz.00067.wav
../data/genres\jazz/jazz.00068.wav
../data/genres\jazz/jazz.00069.wav
../data/genres\jazz/jazz.00070.wav
../data/genres\jazz/jazz.00071.wav
../data/genres\jazz/jazz.00072.wav
../data/genres\jazz/jazz.00073.wav
../data/genres\jazz/jazz.00074.wav
../data/genres\jazz/jazz.00075.wav
../data/genres\jazz/jazz.00076.wav
../data/genres\jazz/jazz.00077.wav
../data/genres\jazz/jazz.00078.wav
../data/genres\jazz/jazz.00079.wav
../data/genres\jazz/jazz.00080.wav
../data/genres\jazz/jazz.00081.wav
../data/genres\jazz/jazz.00082.wav
../data/genres\jazz/jazz.00083.wav
../data/genres\jazz/

In [13]:
entry_clean = []

for entry in entries:
    entry_clean.append([entry[0]] + entry[1])

def get_classical_feature_names(n_mfcc=13, n_chroma=12, n_spec_contrast=7, n_tonnetz=6):
    """
    Returns a list of feature names for the classical features: 
    MFCCs, Chroma, Spectral Contrast, Tonnetz, Tempo.
    """
    column_names = ['target']
    
    # MFCC mean & std
    for i in range(1, n_mfcc+1):
        column_names.append(f"mfcc_{i}_mean")
    for i in range(1, n_mfcc+1):
        column_names.append(f"mfcc_{i}_std")
    
    # Chroma mean & std
    for i in range(1, n_chroma+1):
        column_names.append(f"chroma_{i}_mean")
    for i in range(1, n_chroma+1):
        column_names.append(f"chroma_{i}_std")

    # Spectral contrast mean & std
    for i in range(1, n_spec_contrast+1):
        column_names.append(f"spec_contrast_{i}_mean")
    for i in range(1, n_spec_contrast+1):
        column_names.append(f"spec_contrast_{i}_std")

    # Tonnetz mean & std
    for i in range(1, n_tonnetz+1):
        column_names.append(f"tonnetz_{i}_mean")
    for i in range(1, n_tonnetz+1):
        column_names.append(f"tonnetz_{i}_std")
    
    # Tempo
    column_names.append("tempo")
    column_names.append("mean_spectral_rolloff")
    column_names.append("std_spectral_rolloff")
    column_names.append("mean_spectral_centroids")
    column_names.append("std_spectral_centroids")
    
    # Zero Crossing Rate
    column_names.append("zc_rate")
    
    
    # rms mean and rms var
    column_names.append("rms_mean")
    column_names.append("rms_var")
    
    return column_names

df = pd.DataFrame(np.row_stack(entry_clean), columns=get_classical_feature_names())
df.to_csv("../data/files/audio_classification_5sec.csv", index=False)

In [ ]:
files = get_all_image_files('../data/images/')
mel_spectograms = []

for file in files:
    try:
        filename = file.split('/')[-1]
        print(filename)
        img = Image.open(file).convert('L')  # ensure RGB format
        img_array = np.asarray(img).flatten()
        img_array = img_array.astype(np.float16)
        mel_spectograms.append([filename] + img_array.tolist())
        img.close()
    except Exception as e:
        print(e)
        print(f"Could not process {file}")
        
df = pd.DataFrame.from_records(mel_spectograms)
df.head(10)
df.to_csv("../data/files/mel_spectograms_flat.csv")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import utils

def clean_labels(s):
    return s.split('.')[0]

labelMap = {
    "blues": 0,
    "classical": 1,
    "country": 2,
    "disco": 3,
    "hiphop": 4,
    "jazz": 5,
    "metal": 6,
    "pop": 7,
    "reggae": 8,
    "rock": 9
}

X = df['features']
y = np.array(list(map(lambda x: labelMap[x], df['label'])))
y_cat = utils.to_categorical(y, num_classes=10)

X_train, X_test, y_train, y_test = train_test_split(X, y_cat, random_state=1122)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(pd.concat([X_train_scaled, y_train]), batch_size=8, shuffle=True, drop_last=True)
test_loader  = DataLoader(test_dataset, batch_size=8, shuffle=False, drop_last=False)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=4,
    validation_split=0.2
)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {test_acc:.2%}")

y_pred = model.predict(X_test).argmax(axis=1)
y_true_labels = np.argmax(y_test, axis=1)
print(classification_report(y_true_labels, y_pred))

# Confusion Matrix
cm = confusion_matrix(y_true_labels, y_pred)
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=labels, yticklabels=labels, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()